In [105]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import re
import requests
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

In [106]:
client = MongoClient("localhost:27017")

In [107]:
db = client["Ironhack"]

In [108]:
comp = db.get_collection("companies")

## Getting the address of each company in palo alto

In [131]:
companies_name   = {'name':
                        {"$regex": 
                        '(SurveyMonkey|LivingSocial|Zynga|BrightSource|Better Place|Tesla Motors|SolarCity|Telefonica|Comcast|SunEdison)',
                        "$options" :'i'}}
companies_city   = {'offices.city':'Palo Alto'}
companies_city_1 = {'offices.city':'San Francisco'}                

project_= {'offices.address1':1,'offices.city':1,'_id':0}

In [132]:
address = list(comp.find({ "$and": 
                [companies_name,{'$or':
                                 [companies_city,companies_city_1]}]},project_ ))

In [133]:
address

[{'offices': [{'address1': '3500 Deer Creek rd.', 'city': 'Palo Alto'}]},
 {'offices': [{'address1': '365 Vermont St.', 'city': 'San Francisco'},
   {'address1': '', 'city': ''},
   {'address1': '', 'city': 'Baltimore'},
   {'address1': '', 'city': 'Los Angeles'}]},
 {'offices': [{'address1': '1070 Arastradero Road,', 'city': 'Palo Alto'}]},
 {'offices': [{'address1': '111 SW 5th Ave', 'city': 'Portland'},
   {'address1': '285 Hamilton Avenue', 'city': 'Palo Alto'},
   {'address1': 'Rua Ivens. 3, Edificio D.', 'city': 'Funchal'}]}]

In [134]:
address_lst= []
for i in address:
    for key, value in i.items():
        for j in value:
            address_lst.append(j)

In [135]:
            
df_address= pd.DataFrame(address_lst)

In [136]:
df_address=df_address.apply(lambda x: x[df_address['city'].isin(['Palo Alto'])])

In [137]:
df_address

,address1,city
0,3500 Deer Creek rd.,Palo Alto
5,"1070 Arastradero Road,",Palo Alto
7,285 Hamilton Avenue,Palo Alto


## Coordinates

In [139]:
where= '3500 Deer Creek rd., Palo Alto, United States of America'

In [140]:
url_geocode = f"https://geocode.xyz/{where}?json=1"
requests.get(url_geocode).json()

{'success': False,
 'error': {'code': '006',
  'message': 'Request Throttled. Over Rate limit: up to 2 per sec. See geocode.xyz/pricing',
  'requests': '93'}}

In [147]:
list= [
    {'name':'Tesla','lat':37.3942781,'lon':-122.152006817},
    {'name':'Stanford','lat':37.3942641 ,'lon':-122.14902917 },
    {'name':'City hall ','lat':37.4451016 ,'lon': -122.162978817}
]

In [148]:
df_coordinates= pd.DataFrame(list)

In [149]:
df_coordinates

,name,lat,lon
0,Tesla,37.394278,-122.152007
1,Stanford,37.394264,-122.149029
2,City hall,37.445102,-122.162979
